In [1]:
from collections import defaultdict
import data as dt
import pandas as pd
import numpy as np 
import fair_classification.utils as ut
import json
import os
import math
import data as dt

In [2]:
# stores key value pairs of dataset, sensitive attributes and target labels
DATA2D = {'adult': 'target',
          'compas': 'ScoreText_',
          'german': 'loan_status',
          'synthetic' : 'D'}

DATA2S = {'adult': 'sex',
          'compas': 'Ethnic_Code_Text_',
          'german': 'sex',
          'synthetic': 'S'}

NAMES = ['adult', 'compas', 'german', 'synthetic']

In [3]:
# read input parameters
# debias = true represents proposed method for deriving fair label

config_path = os.path.join("report_input.json")
config = None

with open(config_path, 'r') as fh:
    content = json.load(fh)



dataset = content['dataset']
exp_num = content['exp-id']
num_X = content['num_X']
fold = content['fold']
debias = content['debias']

In [4]:
train_data, test_data, cloumns, decision_label, train_y_fair, train_y_proxy, test_y_fair, test_y_proxy , test_y_debias, train_y_debias= dt.load_data(dataset, fold, num_X=num_X, use_fair=debias, exp_num = exp_num)

# train splits
train_y = train_y_fair if debias else train_data[decision_label]
train_X = train_data.drop(columns=cloumns)
train_sex = train_data[DATA2S[dataset]]
s_train = np.array(train_data[DATA2S[dataset]])

# test splits
test_X = test_data.drop(columns=cloumns)
test_y = test_y_fair if debias else test_data[decision_label]
s_test = np.array(test_data[DATA2S[dataset]])
results = defaultdict(dict)

## EXPERIMENT 1: Unmitigation classification

In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
logisticRegr = LogisticRegression()
logisticRegr.fit(train_X, train_y)

LogisticRegression()

In [7]:
train_pred_prob = logisticRegr.predict_log_proba(train_X)
train_pred = logisticRegr.predict(train_X)
test_pred = logisticRegr.predict(test_X)
test_pred_prob = logisticRegr.predict_log_proba(test_X)

In [8]:
# metrics for this prediction
from fairlearn.metrics import equalized_odds_difference
results["unmitigatedLR"]["fair"] = dict()
results["unmitigatedLR"]["proxy"] = dict()

results["unmitigatedLR"]["fair"]["eo_diff"] = equalized_odds_difference(test_y, test_pred, sensitive_features=s_test)

In [9]:
# eo difference tested against proxy labels
results["unmitigatedLR"]["proxy"]["eo_diff"] = equalized_odds_difference(test_y_proxy, test_pred, sensitive_features=s_test)

In [10]:
from sklearn.metrics import roc_auc_score, f1_score

In [11]:
results["unmitigatedLR"]["fair"]["roc_auc"]= roc_auc_score(test_y, test_pred )
results["unmitigatedLR"]["fair"]["f1"] = f1_score(test_y, test_pred)
results["unmitigatedLR"]["fair"]["check_acc"] = ut.check_accuracy(None, train_X, train_y, test_X, test_y, train_pred, test_pred)


In [12]:
# against proxy label

results["unmitigatedLR"]["proxy"]["roc_auc"] = roc_auc_score(test_y_proxy, test_pred )
results["unmitigatedLR"]["proxy"]["f1"] = f1_score(test_y_proxy, test_pred)

## Exponentiated gradient

In [13]:
import fair_reduction as fr

In [14]:
prob_train, prob_test = fr.reduction(dataset, fold, num_X=num_X, use_debias = debias, exp_num=exp_num)

Utils: check accuracy score
(0.6666666666666666, 0.63, 540, 63)
Equalized odds values:
0.26315789473684215


In [15]:
results["reduction"]["proxy"] = dict()
results["reduction"]["fair"] = dict()

results["reduction"]["fair"]["roc_auc"] = roc_auc_score(test_y, test_pred )
results["reduction"]["fair"]["f1"] = f1_score(test_y, test_pred)
results["reduction"]["fair"]["check_acc"] = ut.check_accuracy(None, train_X, train_y, test_X, test_y, train_pred, test_pred)


In [16]:
results["reduction"]["proxy"]["roc_auc"] = roc_auc_score(test_y_proxy, test_pred )
results["reduction"]["proxy"]["f1"] = f1_score(test_y_proxy, test_pred)
results["reduction"]["proxy"]["check_acc"] = ut.check_accuracy(None, train_X, train_y, test_X, test_y_proxy, train_pred, test_pred)

In [17]:
results["reduction"]["proxy"]["eo_diff"] = equalized_odds_difference(test_y_proxy, np.array(test_pred > 0.5).astype(int), sensitive_features=s_test)

In [18]:
results["reduction"]["fair"]["eo_diff"] = equalized_odds_difference(test_y, np.array(test_pred > 0.5).astype(int), sensitive_features=s_test)

## Fair LR

In [19]:
import fair_lr as flr
import fair_classification.loss_funcs as lf # loss funcs that can be optimized subject to various constraints

In [20]:
SV = DATA2S[dataset]
x_control_train = {SV: s_train}
S_test = np.array(test_data[DATA2S[dataset]])
x_control_test = {SV: s_test}
results["fairLR"]["fair"] = dict()
results["fairLR"]["proxy"] = dict()

In [21]:
def model(x_train, y_train, x_control_train, x_test, y_test, x_control_test, SV):
    x_train = ut.add_intercept(x_train)
    x_test = ut.add_intercept(x_test)
    apply_fairness_constraints = 1
    apply_accuracy_constraint = 0
    sensitive_attrs = [SV]
    sensitive_attrs_to_cov_thresh = {SV: 0}
    sep_constraint = 0
    loss_function = lf._logistic_loss
    gamma = 0
    
    
    def train_test_classifier():
        w = ut.train_model(x_train, y_train, x_control_train, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma)
        train_score, test_score, correct_answers_train, correct_answers_test = ut.check_accuracy(w, x_train, y_train, x_test, y_test, None, None)
        distances_boundary_test = np.dot(x_test, w)
        distances_boundary_train = np.dot(x_train, w)
        prob_test = [dt.sigmoid(x) for x in distances_boundary_test]
        prob_train = [dt.sigmoid(x) for x in distances_boundary_train]
        all_class_labels_assigned_test = np.sign(distances_boundary_test)
        correlation_dict_test = ut.get_correlations(None, None, all_class_labels_assigned_test, x_control_test, sensitive_attrs)
        cov_dict_test = ut.print_covariance_sensitive_attrs(None, x_test, distances_boundary_test, x_control_test, sensitive_attrs)
        p_rule = ut.print_classifier_fairness_stats([test_score], [correlation_dict_test], [cov_dict_test], sensitive_attrs[0])	
        # return w, p_rule, test_score
        return prob_train, prob_test
    return train_test_classifier()

In [22]:

prob_train, prob_test = model(train_X, train_y, x_control_train, test_X, test_y, x_control_test, SV)

dt.save_file(dataset, num_X, fold, "LR", np.array(prob_train), s_train, train_y_fair, train_y_proxy, np.array(prob_test), s_test, test_y_fair, test_y_proxy)


Accuracy: 0.70
Protected/non-protected in +ve class: 100% / 100%
P-rule achieved: 100%
Covariance between sensitive feature and decision from distance boundary : 0.028




In [23]:
results["fairLR"]["fair"]["roc_auc"] = roc_auc_score(test_y, prob_test)
results["fairLR"]["proxy"]["roc_auc"] = roc_auc_score(test_y_proxy, prob_test)

In [24]:
bool_test = (np.array(prob_test) > 0.5).astype(int)

In [25]:
results["fairLR"]["fair"]["eo_diff"] = equalized_odds_difference(test_y, bool_test, sensitive_features=s_test)
results["fairLR"]["proxy"]["eo_diff"] = equalized_odds_difference(test_y_proxy, bool_test, sensitive_features=s_test)

In [26]:
results["fairLR"]["fair"]["f1"] = f1_score(test_y, bool_test)
results["fairLR"]["proxy"]["f1"] = f1_score(test_y_proxy, bool_test)

## Reweigh

In [27]:
import reweight as rw
results["reweigh"]["fair"] = dict()
results["reweigh"]["proxy"] = dict()


In [28]:
prob_train, prob_test = rw.model(dataset, fold, num_X=num_X, use_fair = debias, exp_num = exp_num)

[0.17457577]
Train Accuracy 0.27037037037037037
Train Violation 0.001003393832079147  		 All violations [0.001003393832079147]
Test Accuracy 1.0
Test Violation 0.0485714285714286  		 All violations [-0.0485714285714286]




In [29]:
results["reweigh"]["fair"]["check_acc"] = ut.check_accuracy(None, train_X, train_y, test_X, test_y, prob_train, prob_test)

In [30]:
bool_test = (prob_test > 0.5).astype(int)

In [31]:
results["reweigh"]["fair"]["eo_diff"] = equalized_odds_difference(test_y, bool_test, sensitive_features=s_test)
results["reweigh"]["proxy"]["eo_diff"] = equalized_odds_difference(test_y_proxy, bool_test, sensitive_features=s_test)

In [32]:
results["reweigh"]["fair"]["f1"] = f1_score(test_y, bool_test)
results["reweigh"]["proxy"]["f1"] = f1_score(test_y_proxy, bool_test)

In [33]:
print(results)

defaultdict(<class 'dict'>, {'unmitigatedLR': {'fair': {'eo_diff': 0.1722488038277512, 'roc_auc': 0.5357142857142857, 'f1': 0.7870967741935484, 'check_acc': (0.7308641975308642, 0.67, 592, 67)}, 'proxy': {'eo_diff': 0.1722488038277512, 'roc_auc': 0.5357142857142857, 'f1': 0.7870967741935484}}, 'reduction': {'proxy': {'roc_auc': 0.5357142857142857, 'f1': 0.7870967741935484, 'check_acc': (0.7308641975308642, 0.67, 592, 67), 'eo_diff': 0.1722488038277512}, 'fair': {'roc_auc': 0.5357142857142857, 'f1': 0.7870967741935484, 'check_acc': (0.7308641975308642, 0.67, 592, 67), 'eo_diff': 0.1722488038277512}}, 'fairLR': {'fair': {'roc_auc': 0.5504761904761905, 'eo_diff': 0.0, 'f1': 0.8235294117647058}, 'proxy': {'roc_auc': 0.5504761904761905, 'eo_diff': 0.0, 'f1': 0.8235294117647058}}, 'reweigh': {'fair': {'check_acc': (0.0, 0.0, 0, 0), 'eo_diff': 0.1722488038277512, 'f1': 0.810126582278481}, 'proxy': {'eo_diff': 0.1722488038277512, 'f1': 0.810126582278481}}})
